In [10]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
import time

In [6]:
# amazon website
url = "https://www.amazon.com/s?k=led&ref=nb_sb_noss_2"

header = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
          'cookie': 'session-id=136-6766982-5860004; session-id-time=2082787201l; sp-cdn="L5Z9:TW"; ubid-main=130-9209971-5955330; session-token=pVazN62vPRjiuDjN45tA43nH8DiNXYfcOa99xhYEXA0U/x6ulYkLuKutM1jiMfY83QABeddgRmIzYA1vEh0QsWUm9QeEJeJaHzgT8DHoQnxA8jR+YuQa6vilTZHZ9smiF9JMRTx8OpQzupWSQxEZA7VadmyixUmi5klaRxWzN+8mcr8V7ljJ8wTHUHxhyhqt; lc-main=en_US; i18n-prefs=USD; csm-hit=tb:ZK67JQSZ17TSG8PPFBZ5+s-17GRW1S7EWCY3AF68K80|1630561263645&t:1630561263645&adb:adblk_yes'}

ss = requests.session()
res = ss.get(url, headers=header)

In [7]:
soup = BeautifulSoup(res.text, features='html.parser')

In [16]:
url = "https://www.amazon.com/s?k=led&ref=nb_sb_noss_2"
driver = Chrome("../chromedriver")
driver.get(url)
time.sleep(2)

# 取得頁面的html
soup = BeautifulSoup(driver.page_source, "html.parser")

driver.close()

In [ ]:
# 總共77個
product_info = soup.select("div[class='a-section a-spacing-none']")

In [64]:
product_info[0]

<div class="a-section a-spacing-none"><div class="a-section a-spacing-none a-spacing-top-small"><div class="a-row a-spacing-micro"><span class="a-declarative" data-a-popover='{"closeButton":"true","dataStrategy":"preload","name":"sp-info-popover-mobile"}' data-action="a-popover"><span class="s-sponsored-label-text"><span class="a-color-secondary">贊助</span></span></span><div class="a-popover-preload" id="a-popover-sp-info-popover-mobile"><span class="a-color-secondary">贊助</span></div></div><h2 class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"><a class="a-link-normal a-text-normal" href="/-/zh_TW/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&amp;adId=A00448563RG9K8SPYSBWC&amp;url=%2FLepro-LED-%25E5%25B8%25B6%25E9%2581%2599%25E6%258E%25A7%25E5%2599%25A8%25E5%2592%258C%25E5%259B%25BA%25E5%25AE%259A%25E5%25A4%25BE-%25E8%25AE%258A%25E8%2589%25B2%25E8%2586%25A0%25E5%25B8%25B6%25E7%2587%2588-%25E9%2581%25A9%25E7%2594%25A8%25E6%2596%25BC%25E8%2587%25A5%25E5%

In [62]:
# 取得產品名稱
title = product_info[0].select("span[class='a-size-base-plus a-color-base a-text-normal']")[0].text
title

'Lepro 50 英尺(約 15.2 公尺)LED 條燈,超長 RGB 5050 LED 條帶,帶遙控器和固定夾,變色膠帶燈,附 24V ETL 認證適配器,適用於臥室、房間、廚房、酒吧(2 X 24.6 呎)'

In [81]:
# 取得價格
price = product_info[0].select(".a-offscreen")[0].text   #("span[class='a-price'] > span[class='a-offscreen']")[0].text
price

'US$21.99'

In [83]:
# 取得星星＆評分數
star, rated_amount = [i.text for i in product_info[0].select("div[class='a-row a-size-small'] > span")]
star, rated_amount

('4.6 顆星，最高 5 顆星 ', '14,342  ')